# Piper TTS Benchmark

Bu notebook Piper TTS engine'ini 5 Türkçe mülakat sorusu ile test eder.

**Engine:** Piper (CPU-only)
**GPU:** Gerekmez
**Türkçe Desteği:** Evet (tr_TR-dfki-medium model)

Her soru için hem SORU hem CEVAP seslendiriliyor.

In [ ]:
# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("Drive mounted successfully")

In [ ]:
# Cell 2: Install Dependencies
print("Installing dependencies...")

!pip install piper-tts

import os
TTS_MS_PATH = "/content/drive/MyDrive/tts-ms"
if not os.path.exists(TTS_MS_PATH):
    raise FileNotFoundError(f"tts-ms not found at {TTS_MS_PATH}")

!pip install "/content/drive/MyDrive/tts-ms"

import tts_ms
print(f"tts_ms installed: {tts_ms.__file__}")

In [ ]:
# Cell 3: Setup & Download Model
import os
from datetime import datetime

ENGINE_NAME = "piper"
MODEL_NAME = "tr_TR-dfki-medium"

BASE_DIR = f"/content/drive/MyDrive/tts-ms/output/{ENGINE_NAME}"
AUDIO_DIR = f"{BASE_DIR}/audio"

# Temiz baslangic
import shutil
if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)
os.makedirs(AUDIO_DIR, exist_ok=True)

print(f"Output: {BASE_DIR}")

os.environ["TTS_MODEL_TYPE"] = ENGINE_NAME
os.environ["TTS_MS_LOG_LEVEL"] = "4"
os.environ["TTS_MS_RESOURCES_ENABLED"] = "1"

# Model indir
MODEL_DIR = "/content/drive/MyDrive/tts-ms/cache/piper"
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATH = f"{MODEL_DIR}/{MODEL_NAME}.onnx"
CONFIG_PATH = f"{MODEL_DIR}/{MODEL_NAME}.onnx.json"

if not os.path.exists(MODEL_PATH):
    print(f"Downloading {MODEL_NAME}...")
    !wget -q -O "{MODEL_PATH}" "https://huggingface.co/rhasspy/piper-voices/resolve/main/tr/tr_TR/dfki/medium/tr_TR-dfki-medium.onnx"
    !wget -q -O "{CONFIG_PATH}" "https://huggingface.co/rhasspy/piper-voices/resolve/main/tr/tr_TR/dfki/medium/tr_TR-dfki-medium.onnx.json"
    print("Model downloaded.")
else:
    print(f"Model cached: {MODEL_PATH}")

In [ ]:
# Cell 4: Initialize TTSService
import time
import psutil
from tts_ms.services import TTSService
from tts_ms.services.tts_service import SynthesizeRequest
from tts_ms.core.config import Settings

process = psutil.Process()
CPU_COUNT = psutil.cpu_count()
resource_logs = []

def get_resources():
    return {'cpu': process.cpu_percent(), 'ram_mb': process.memory_info().rss / 1024 / 1024}

print("Initializing TTSService...")
res_before = get_resources()
start = time.time()

settings = Settings(raw={
    'tts': {
        'engine': 'piper',
        'device': 'cpu',
        'piper': {'model_path': MODEL_PATH, 'config_path': CONFIG_PATH}
    },
    'cache': {'enabled': False},
    'storage': {'enabled': False},
    'logging': {'level': 4}
})
service = TTSService(settings)
init_time = time.time() - start
res_after = get_resources()
resource_logs.append({'stage': 'init', 'duration': init_time, 'cpu': res_after['cpu'], 'ram_delta': res_after['ram_mb'] - res_before['ram_mb']})
print(f"Initialized in {init_time:.2f}s")

# Warmup
print("\nWarmup...")
res_before = get_resources()
start = time.time()
_ = service.synthesize(SynthesizeRequest(text="Merhaba."), request_id="warmup")
warmup_time = time.time() - start
res_after = get_resources()
resource_logs.append({'stage': 'warmup', 'duration': warmup_time, 'cpu': res_after['cpu'], 'ram_delta': res_after['ram_mb'] - res_before['ram_mb']})
print(f"Warmup done in {warmup_time:.2f}s")

In [ ]:
# Cell 5: Synthesize
from IPython.display import Audio, display

QUESTIONS = [
    {"id": "01", "question": "Sizi neden işe almalıyız?",
     "answer": "Güçlü analitik düşünme becerilerim ve takım çalışmasına yatkınlığım sayesinde projelere değer katabilirim. Ayrıca sürekli öğrenmeye açık yapım ve problem çözme yeteneklerim, şirketinizin hedeflerine ulaşmasında önemli katkılar sağlayacaktır."},
    {"id": "02", "question": "Siz bizi neden seçtiniz?",
     "answer": "Şirketinizin yenilikçi yaklaşımı ve sektördeki lider konumu beni çok etkiledi. Kariyer hedeflerimle örtüşen bu ortamda kendimi geliştirebileceğime ve anlamlı projeler üzerinde çalışabileceğime inanıyorum."},
    {"id": "03", "question": "Kötü özellikleriniz nelerdir?",
     "answer": "Bazen aşırı detaycı olabiliyorum, bu da zaman yönetimimi olumsuz etkileyebiliyor. Ancak bu özelliğimin farkındayım ve önceliklendirme teknikleri kullanarak bu durumu yönetmeye çalışıyorum."},
    {"id": "04", "question": "Beş yıl sonra kendinizi nerede görüyorsunuz?",
     "answer": "Beş yıl içinde teknik liderlik pozisyonunda olmayı hedefliyorum. Ekip yönetimi deneyimi kazanarak şirketin büyümesine stratejik katkılar sağlamak istiyorum."},
    {"id": "05", "question": "Maaş beklentiniz nedir?",
     "answer": "Piyasa koşullarını ve pozisyonun gerekliliklerini değerlendirerek, deneyimime ve yeteneklerime uygun rekabetçi bir maaş beklentim var. Bu konuda esnek olmaya ve karşılıklı bir anlaşmaya varmaya açığım."}
]

results = []
print(f"{'='*60}")
print(f"SYNTHESIZING {len(QUESTIONS)} QUESTIONS + ANSWERS")
print(f"{'='*60}\n")

for q in QUESTIONS:
    print(f"\n[{q['id']}] {q['question']}")
    
    for typ, text in [('soru', q['question']), ('cevap', q['answer'])]:
        print(f"  {typ.upper()}: ", end="", flush=True)
        res_before = get_resources()
        start = time.time()
        try:
            result = service.synthesize(SynthesizeRequest(text=text), request_id=f"{q['id']}_{typ}")
            elapsed = time.time() - start
            res_after = get_resources()
            
            path = f"{AUDIO_DIR}/{q['id']}_{typ}.wav"
            with open(path, 'wb') as f:
                f.write(result.wav_bytes)
            
            cpu_norm = res_after['cpu'] / CPU_COUNT
            ram_delta = res_after['ram_mb'] - res_before['ram_mb']
            
            resource_logs.append({'stage': f"{q['id']}_{typ}", 'duration': elapsed, 'cpu': res_after['cpu'], 'cpu_norm': cpu_norm, 'ram_delta': ram_delta, 'size_kb': len(result.wav_bytes)/1024})
            results.append({'id': q['id'], 'type': typ, 'time': elapsed, 'size': len(result.wav_bytes), 'cpu': cpu_norm, 'ram_delta': ram_delta, 'status': 'OK'})
            
            print(f"{elapsed:.2f}s | {len(result.wav_bytes)/1024:.1f} KB | CPU:{cpu_norm:.0f}% | OK")
            display(Audio(path))
        except Exception as e:
            results.append({'id': q['id'], 'type': typ, 'time': time.time()-start, 'size': 0, 'cpu': 0, 'ram_delta': 0, 'status': f'FAIL: {e}'})
            print(f"FAIL: {e}")

successful = [r for r in results if r['status'] == 'OK']
print(f"\n{'='*60}")
print(f"COMPLETE: {len(successful)}/{len(results)} successful")
print(f"{'='*60}")

In [ ]:
# Cell 6: Generate Reports
import json
from datetime import datetime

successful = [r for r in results if r['status'] == 'OK']
soru_results = [r for r in successful if r['type'] == 'soru']
cevap_results = [r for r in successful if r['type'] == 'cevap']

total_size_kb = sum(r['size'] for r in successful) / 1024
avg_soru = sum(r['time'] for r in soru_results) / len(soru_results) if soru_results else 0
avg_cevap = sum(r['time'] for r in cevap_results) / len(cevap_results) if cevap_results else 0

synth_logs = [r for r in resource_logs if r['stage'] not in ['init', 'warmup']]
avg_cpu = sum(r.get('cpu_norm', 0) for r in synth_logs) / len(synth_logs) if synth_logs else 0
max_cpu = max(r.get('cpu_norm', 0) for r in synth_logs) if synth_logs else 0
total_ram = sum(r.get('ram_delta', 0) for r in synth_logs)

# summary.txt
summary = f"""============================================================
TTS-MS BENCHMARK - PIPER
============================================================
Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Platform: Google Colab (CPU - {CPU_COUNT} core)
Model: {MODEL_NAME}

PERFORMANS:
- Init: {init_time:.2f}s
- Warmup: {warmup_time:.2f}s

SONUCLAR:
- Toplam: {len(QUESTIONS)} soru
- Soru Audio: {len(soru_results)}/5 basarili (ort: {avg_soru:.2f}s)
- Cevap Audio: {len(cevap_results)}/5 basarili (ort: {avg_cevap:.2f}s)
- Toplam Audio: {total_size_kb:.1f} KB

KAYNAK KULLANIMI:
- Ortalama CPU: {avg_cpu:.0f}%
- Maksimum CPU: {max_cpu:.0f}%
- RAM Delta: {total_ram:+.1f} MB
- GPU: N/A (CPU-only model)

DETAYLAR:
"""
for r in results:
    s = 'OK' if r['status'] == 'OK' else 'FAIL'
    summary += f"[{r['id']}_{r['type']}] {r['time']:.2f}s | {r['size']/1024:.1f} KB | CPU:{r['cpu']:.0f}% | {s}\n"
summary += f"\nAudio: {AUDIO_DIR}\n============================================================\n"

with open(f"{BASE_DIR}/summary.txt", 'w', encoding='utf-8') as f:
    f.write(summary)

# resource.log
res_log = f"""============================================================
TTS-MS PIPER - RESOURCE LOG
============================================================
Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Platform: Google Colab (CPU - {CPU_COUNT} core)
Model: {MODEL_NAME}

KAYNAK KULLANIMI (ADIM ADIM):
============================================================\n"""
for r in resource_logs:
    if r['stage'] == 'init':
        res_log += f"[INIT] {r['duration']:.2f}s | RAM: {r['ram_delta']:+.1f} MB\n"
    elif r['stage'] == 'warmup':
        res_log += f"[WARMUP] {r['duration']:.2f}s | CPU: {r['cpu']/CPU_COUNT:.0f}% | RAM: {r['ram_delta']:+.1f} MB\n"
    else:
        res_log += f"[{r['stage'].upper()}] {r['duration']:.2f}s | {r.get('size_kb',0):.1f} KB | CPU: {r.get('cpu_norm',0):.0f}% | RAM: {r['ram_delta']:+.1f} MB\n"
res_log += f"""\n============================================================
OZET:
- Ortalama CPU: {avg_cpu:.0f}%
- Maksimum CPU: {max_cpu:.0f}%
- Toplam RAM Delta: {total_ram:+.1f} MB
- Ortalama Synth: {(avg_soru+avg_cevap)/2:.2f}s
============================================================\n"""

with open(f"{BASE_DIR}/resource.log", 'w', encoding='utf-8') as f:
    f.write(res_log)

# results.json
with open(f"{BASE_DIR}/results.json", 'w', encoding='utf-8') as f:
    json.dump({'engine': ENGINE_NAME, 'model': MODEL_NAME, 'results': results, 'resources': resource_logs,
               'init_time': init_time, 'warmup_time': warmup_time, 'avg_cpu': avg_cpu, 'max_cpu': max_cpu}, f, indent=2, ensure_ascii=False)

print(summary)
print("\n" + res_log)

In [ ]:
# Cell 7: Download ZIP
import zipfile
from pathlib import Path
from google.colab import files

zip_path = f"{BASE_DIR}/piper_benchmark.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    zf.write(f"{BASE_DIR}/summary.txt", "summary.txt")
    zf.write(f"{BASE_DIR}/resource.log", "resource.log")
    zf.write(f"{BASE_DIR}/results.json", "results.json")
    for wav in Path(AUDIO_DIR).glob("*.wav"):
        zf.write(wav, f"audio/{wav.name}")

print(f"ZIP: {zip_path}")
print(f"Size: {Path(zip_path).stat().st_size/1024:.1f} KB")
print("\nContents:")
with zipfile.ZipFile(zip_path, 'r') as zf:
    for f in zf.namelist():
        print(f"  {f}")

files.download(zip_path)